# Analisi e Previsione della Produzione di Olio d'Oliva

Questo notebook esplora la relazione tra i dati meteorologici e la produzione annuale di olio d'oliva, con l'obiettivo di creare un modello predittivo.

In [ ]:
import os

# Rimuove il file se esiste
if os.path.exists('output.zip'):
    os.remove('output.zip')
    
!zip -r output.zip /kaggle/working/

In [ ]:
import tensorflow as tf

print(f"Keras version: {tf.keras.__version__}")
print(f"TensorFlow version: {tf.__version__}")

# GPU configuration
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

In [ ]:
# Test semplice per verificare che la GPU funzioni
def test_gpu():
    print("TensorFlow version:", tf.__version__)
    print("\nDispositivi disponibili:")
    print(tf.config.list_physical_devices())

    # Creiamo e moltiplichiamo due tensori sulla GPU
    with tf.device('/GPU:0'):
        a = tf.random.normal([10000, 10000])
        b = tf.random.normal([10000, 10000])
        c = tf.matmul(a, b)

    print("\nShape del risultato:", c.shape)
    print("Device del tensore:", c.device)
    return "Test completato con successo!"


test_gpu()

In [ ]:
!pip install numpy
!pip install pandas

!pip install keras
!pip install scikit-learn
!pip install matplotlib
!pip install joblib
!pip install pyarrow
!pip install fastparquet
!pip install scipy
!pip install seaborn
!pip install tqdm

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.keras.layers import Input, Dense, Dropout, Bidirectional, LSTM, LayerNormalization, Add, Activation, BatchNormalization, MultiHeadAttention, MaxPooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import mean_squared_error, mean_absolute_error
from datetime import datetime
import os
import json
import joblib
import re
import pyarrow as pa
import pyarrow.parquet as pq
from tqdm import tqdm

random_state_value = 42

## Funzioni di Plot

In [ ]:
def save_plot(plt, title, output_dir='/kaggle/working/plots'):
    os.makedirs(output_dir, exist_ok=True)
    filename = "".join(x for x in title if x.isalnum() or x in [' ', '-', '_']).rstrip()
    filename = filename.replace(' ', '_').lower()
    filepath = os.path.join(output_dir, f"{filename}.png")
    plt.savefig(filepath, bbox_inches='tight', dpi=300)
    print(f"Plot salvato come: {filepath}")

## 1. Caricamento e preparazione dei Dati Meteo

In [ ]:
# Function to convert csv to parquet
def csv_to_parquet(csv_file, parquet_file, chunksize=100000):
    writer = None

    for chunk in pd.read_csv(csv_file, chunksize=chunksize):
        if writer is None:

            table = pa.Table.from_pandas(chunk)
            writer = pq.ParquetWriter(parquet_file, table.schema)
        else:
            table = pa.Table.from_pandas(chunk)

        writer.write_table(table)

    if writer:
        writer.close()

    print(f"File conversion completed : {csv_file} -> {parquet_file}")


def read_json_files(folder_path):
    all_data = []

    file_list = sorted(os.listdir(folder_path))

    for filename in file_list:
        if filename.endswith('.json'):
            file_path = os.path.join(folder_path, filename)
            try:
                with open(file_path, 'r') as file:
                    data = json.load(file)
                    all_data.extend(data['days'])
            except Exception as e:
                print(f"Error processing file '{filename}': {str(e)}")

    return all_data


def create_weather_dataset(data):
    dataset = []
    seen_datetimes = set()

    for day in data:
        date = day['datetime']
        for hour in day['hours']:
            datetime_str = f"{date} {hour['datetime']}"

            # Verifico se questo datetime è già stato visto
            if datetime_str in seen_datetimes:
                continue

            seen_datetimes.add(datetime_str)

            if isinstance(hour['preciptype'], list):
                preciptype = "__".join(hour['preciptype'])
            else:
                preciptype = hour['preciptype'] if hour['preciptype'] else ""

            conditions = hour['conditions'].replace(', ', '__').replace(' ', '_').lower()

            row = {
                'datetime': datetime_str,
                'temp': hour['temp'],
                'feelslike': hour['feelslike'],
                'humidity': hour['humidity'],
                'dew': hour['dew'],
                'precip': hour['precip'],
                'snow': hour['snow'],
                'preciptype': preciptype.lower(),
                'windspeed': hour['windspeed'],
                'winddir': hour['winddir'],
                'pressure': hour['pressure'],
                'cloudcover': hour['cloudcover'],
                'visibility': hour['visibility'],
                'solarradiation': hour['solarradiation'],
                'solarenergy': hour['solarenergy'],
                'uvindex': hour['uvindex'],
                'conditions': conditions,
                'tempmax': day['tempmax'],
                'tempmin': day['tempmin'],
                'precipprob': day['precipprob'],
                'precipcover': day['precipcover']
            }
            dataset.append(row)

    dataset.sort(key=lambda x: datetime.strptime(x['datetime'], "%Y-%m-%d %H:%M:%S"))

    return pd.DataFrame(dataset)


folder_path = './data/weather'
#raw_data = read_json_files(folder_path)
#weather_data = create_weather_dataset(raw_data)
#weather_data['datetime'] = pd.to_datetime(weather_data['datetime'], errors='coerce')
#weather_data['date'] = weather_data['datetime'].dt.date
#weather_data = weather_data.dropna(subset=['datetime'])
#weather_data['datetime'] = pd.to_datetime(weather_data['datetime'])
#weather_data['year'] = weather_data['datetime'].dt.year
#weather_data['month'] = weather_data['datetime'].dt.month
#weather_data['day'] = weather_data['datetime'].dt.day
#weather_data.head()

#weather_data.to_parquet('./data/weather_data.parquet')

In [ ]:
# Crea le sequenze per LSTM
def create_sequences(timesteps, X, y=None):
    """
    Crea sequenze temporali dai dati.
    
    Parameters:
    -----------
    X : array-like
        Dati di input
    timesteps : int
        Numero di timestep per ogni sequenza
    y : array-like, optional
        Target values. Se None, crea sequenze solo per X
        
    Returns:
    --------
    tuple o array
        Se y è fornito: (X_sequences, y_sequences)
        Se y è None: X_sequences
    """
    Xs = []
    for i in range(len(X) - timesteps):
        Xs.append(X[i:i + timesteps])

    if y is not None:
        ys = []
        for i in range(len(X) - timesteps):
            ys.append(y[i + timesteps])
        return np.array(Xs), np.array(ys)

    return np.array(Xs)


# Funzioni per costruire il modello LSTM avanzato
def create_residual_lstm_layer(x, units, dropout_rate, l2_reg=0.01, return_sequences=True):
    residual = x
    x = Bidirectional(LSTM(units, return_sequences=return_sequences, kernel_regularizer=l2(l2_reg)))(x)
    x = LayerNormalization()(x)
    x = Dropout(dropout_rate)(x)
    # Adjust residual dimension and handle return_sequences
    if return_sequences:
        if int(residual.shape[-1]) != 2 * units:
            residual = Dense(2 * units, activation='linear')(residual)
        x = Add()([x, residual])
    return x


def attention_block(x, units, num_heads=8):
    attention = MultiHeadAttention(num_heads=num_heads, key_dim=units)(x, x)
    x = Add()([x, attention])
    x = LayerNormalization()(x)
    return x


def build_advanced_model(input_shape, l2_lambda=0.005):
    inputs = Input(shape=input_shape)

    # Primi due layer LSTM con sequenze
    x = create_residual_lstm_layer(inputs, 64, 0.2, l2_lambda, return_sequences=True)
    x = create_residual_lstm_layer(x, 32, 0.2, l2_lambda, return_sequences=True)

    # Attention e MaxPooling mentre abbiamo ancora la sequenza
    x = attention_block(x, 32, num_heads=8)
    x = MaxPooling1D()(x)

    # Ultimo layer LSTM senza sequenze
    x = create_residual_lstm_layer(x, 16, 0.1, l2_lambda, return_sequences=False)

    # Dense layers
    x = Dense(32, kernel_regularizer=l2(l2_lambda))(x)
    x = BatchNormalization()(x)
    x = Activation('swish')(x)
    x = Dropout(0.1)(x)

    x = Dense(16, kernel_regularizer=l2(l2_lambda))(x)
    x = BatchNormalization()(x)
    x = Activation('swish')(x)
    x = Dropout(0.1)(x)

    outputs = Dense(1, kernel_regularizer=l2(l2_lambda))(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model


def get_season(date):
    month = date.month
    day = date.day
    if (month == 12 and day >= 21) or (month <= 3 and day < 20):
        return 'Winter'
    elif (month == 3 and day >= 20) or (month <= 6 and day < 21):
        return 'Spring'
    elif (month == 6 and day >= 21) or (month <= 9 and day < 23):
        return 'Summer'
    elif (month == 9 and day >= 23) or (month <= 12 and day < 21):
        return 'Autumn'
    else:
        return 'Unknown'


def get_time_period(hour):
    if 5 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 17:
        return 'Afternoon'
    elif 17 <= hour < 21:
        return 'Evening'
    else:
        return 'Night'


def add_time_features(df):
    df['datetime'] = pd.to_datetime(df['datetime'])
    df['timestamp'] = df['datetime'].astype(np.int64) // 10 ** 9
    df['year'] = df['datetime'].dt.year
    df['month'] = df['datetime'].dt.month
    df['day'] = df['datetime'].dt.day
    df['hour'] = df['datetime'].dt.hour
    df['minute'] = df['datetime'].dt.minute
    df['hour_sin'] = np.sin(df['hour'] * (2 * np.pi / 24))
    df['hour_cos'] = np.cos(df['hour'] * (2 * np.pi / 24))
    df['day_of_week'] = df['datetime'].dt.dayofweek
    df['day_of_year'] = df['datetime'].dt.dayofyear
    df['week_of_year'] = df['datetime'].dt.isocalendar().week.astype(int)
    df['quarter'] = df['datetime'].dt.quarter
    df['is_month_end'] = df['datetime'].dt.is_month_end.astype(int)
    df['is_quarter_end'] = df['datetime'].dt.is_quarter_end.astype(int)
    df['is_year_end'] = df['datetime'].dt.is_year_end.astype(int)
    df['month_sin'] = np.sin(df['month'] * (2 * np.pi / 12))
    df['month_cos'] = np.cos(df['month'] * (2 * np.pi / 12))
    df['day_of_year_sin'] = np.sin(df['day_of_year'] * (2 * np.pi / 365.25))
    df['day_of_year_cos'] = np.cos(df['day_of_year'] * (2 * np.pi / 365.25))
    df['season'] = df['datetime'].apply(get_season)
    df['time_period'] = df['hour'].apply(get_time_period)
    return df


# Carica il dataset
weather_data = pd.read_parquet('/kaggle/input/olive-oil/weather_data.parquet')

# Aggiungi le caratteristiche temporali
weather_data = add_time_features(weather_data)

# Encoding delle variabili categoriali
weather_data = pd.get_dummies(weather_data, columns=['season', 'time_period'], drop_first=True)

weather_data.to_parquet('/kaggle/working/weather_data_extended.parquet')

# Dividi i dati in quelli dopo il 2010 e quelli prima del 2010
data_after_2010 = weather_data[weather_data['year'] >= 2010].copy()
data_before_2010 = weather_data[weather_data['year'] < 2010].copy()

# Aggiorna le target variables se necessario
target_variables = ['solarradiation', 'solarenergy', 'uvindex']

# Seleziona le features
features = [
               'temp', 'tempmin', 'tempmax', 'humidity', 'cloudcover', 'windspeed', 'pressure', 'visibility',
               'hour_sin', 'hour_cos', 'month_sin', 'month_cos', 'day_of_year_sin', 'day_of_year_cos',
           ] + [col for col in weather_data.columns if 'season_' in col or 'time_period_' in col]

# Prepara data_after_2010
data_after_2010 = data_after_2010.sort_values('datetime')
data_after_2010.set_index('datetime', inplace=True)

# Interpola eventuali valori mancanti nelle variabili target
columns_to_interpolate = target_variables
for column in columns_to_interpolate:
    data_after_2010[column] = data_after_2010[column].interpolate(method='time')

# Rimuovi eventuali valori mancanti residui
data_after_2010.dropna(subset=features + target_variables, inplace=True)

# Crea X e y
X = data_after_2010[features].values
y = data_after_2010[target_variables].values

# Normalizza le features
scaler_X = MinMaxScaler()
X_scaled = scaler_X.fit_transform(X)


def prepare_multi_target_datasets(X_scaled, y, target_variables):
    """
    Prepara dataset separati per ogni target variable e restituisce anche gli scaler
    per un uso successivo in fase di predizione.
    
    Parameters:
    -----------
    X_scaled : numpy.ndarray
        Features già scalate
    y : numpy.ndarray
        Target variables (matrice con una colonna per ogni target)
    target_variables : list
        Lista dei nomi delle variabili target
    
    Returns:
    --------
    tuple (dict, dict)
        - Primo dict: contiene i dataset per ogni target
        - Secondo dict: contiene gli scaler per ogni target
    """

    # Inizializza i dizionari per contenere i dataset e gli scaler
    train_datasets = {}
    scalers_dict = {}

    # Scala e splitta i dati per ogni target
    for i, target in enumerate(target_variables):
        # Scala il target corrente
        scaler = MinMaxScaler()
        y_scaled_current = scaler.fit_transform(y[:, i].reshape(-1, 1)).flatten()
        scalers_dict[target] = scaler

        # Split dei dati per il target corrente
        X_train_full, X_test, y_train_full, y_test = train_test_split(
            X_scaled,
            y_scaled_current,
            test_size=0.2,
            shuffle=False
        )

        # Ulteriore split per validation
        X_train, X_val, y_train, y_val = train_test_split(
            X_train_full,
            y_train_full,
            test_size=0.2,
            shuffle=False
        )

        # Salva i dataset per questo target
        train_datasets[target] = {
            'X_train': X_train,
            'X_val': X_val,
            'X_test': X_test,
            'y_train': y_train.reshape(-1, 1),
            'y_val': y_val.reshape(-1, 1),
            'y_test': y_test.reshape(-1, 1)
        }

    return train_datasets, scalers_dict


datasets, scalers = prepare_multi_target_datasets(X_scaled, y, target_variables)

In [ ]:
# numero di timesteps (utilizziamo le ultime 24 ore)
timesteps = 24

# Costruisci il modello per ogni variabile target
models = {}
histories = {}
for i, target in enumerate(target_variables):
    target_data = datasets[target]
    target_scaler = scalers[target]

    X_train = target_data['X_train']
    y_train = target_data['y_train']
    X_val = target_data['X_val']
    y_val = target_data['y_val']
    X_test = target_data['X_test']
    y_test = target_data['y_test']

    num_features = X_train.shape[1]

    X_train_seq, y_train_seq = create_sequences(timesteps, X_train, y_train)
    X_val_seq, y_val_seq = create_sequences(timesteps, X_val, y_val)
    X_test_seq, y_test_seq = create_sequences(timesteps, X_test, y_test)

    print(X_train_seq.shape, y_train_seq.shape)
    print(X_val_seq.shape, y_val_seq.shape)
    print(X_test_seq.shape, y_test_seq.shape)

    print(f"Addestramento del modello per: {target}")
    model = build_advanced_model((timesteps, num_features), l2_lambda=0.001)
    optimizer = Adam(learning_rate=0.001, clipnorm=1.0)
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-6
    )
    
    
    history = model.fit(
        X_train_seq, y_train_seq,
        validation_data=(X_val_seq, y_val_seq),
        epochs=50,
        batch_size=180,
        callbacks=[
            early_stopping,
            reduce_lr,
            # Model Checkpoint
            tf.keras.callbacks.ModelCheckpoint(
                filepath='/kaggle/working/{target}/best_model_{epoch:02d}_{val_loss:.4f}.keras',
                monitor='val_loss',
                save_best_only=True,
                mode='min'
            ),
            # TensorBoard logging
            tf.keras.callbacks.TensorBoard(
                log_dir='/kaggle/working/{target}/logs',
                histogram_freq=1,
                write_graph=True,
                update_freq='epoch'
            )],
        verbose=1
    )
    test_loss = model.evaluate(X_test_seq, y_test_seq)
    mse, mae = test_loss
    print(f'Test MSE per {target}: {mse:.4f}')
    print(f'Test MAE per {target}: {mae:.4f}')
    models[target] = model
    histories[target] = history


In [ ]:
def save_models_and_scalers(models, scaler_X, scalers_y, target_variables, base_path='/kaggle/working/models'):
    """
    Salva i modelli e gli scaler nella cartella models.
    
    Parameters:
    -----------
    models : dict
        Dizionario contenente i modelli per ogni variabile target
    scaler_X : MinMaxScaler
        Scaler unico per tutte le feature di input
    scalers_y : dict
        Dizionario contenente gli scaler per le variabili target
    target_variables : list
        Lista delle variabili target
    base_path : str
        Percorso base dove salvare i modelli (default: 'models')
    """

    # Crea la cartella se non esiste
    os.makedirs(base_path, exist_ok=True)

    # Salva lo scaler X generale
    scaler_x_path = os.path.join(base_path, 'scaler_x.joblib')
    joblib.dump(scaler_X, scaler_x_path)

    # Salva i modelli e gli scaler Y per ogni variabile target
    for target in target_variables:
        # Crea una sottocartella per ogni target
        target_path = os.path.join(base_path, target)
        os.makedirs(target_path, exist_ok=True)

        # Salva il modello
        model_path = os.path.join(target_path, 'model.joblib')
        joblib.dump(models[target], model_path)

        # Salva lo scaler Y
        scaler_y_path = os.path.join(target_path, 'scaler_y.joblib')
        joblib.dump(scalers_y[target], scaler_y_path)

    # Salva la lista delle variabili target
    target_vars_path = os.path.join(base_path, 'target_variables.joblib')
    joblib.dump(target_variables, target_vars_path)

    print(f"Modelli e scaler salvati in: {base_path}")


def load_models_and_scalers(base_path='/kaggle/working/models'):
    """
    Carica i modelli e gli scaler dalla cartella models.
    
    Parameters:
    -----------
    base_path : str
        Percorso della cartella contenente i modelli salvati (default: 'models')
    
    Returns:
    --------
    tuple
        (models, scaler_X, scalers_y, target_variables)
    """

    # Carica la lista delle variabili target
    target_vars_path = os.path.join(base_path, 'target_variables.joblib')
    target_variables = joblib.load(target_vars_path)

    # Carica lo scaler X generale
    scaler_x_path = os.path.join(base_path, 'scaler_x.joblib')
    scaler_X = joblib.load(scaler_x_path)

    # Inizializza i dizionari
    models = {}
    scalers_y = {}

    # Carica i modelli e gli scaler per ogni variabile target
    for target in target_variables:
        target_path = os.path.join(base_path, target)

        # Carica il modello
        model_path = os.path.join(target_path, 'model.joblib')
        models[target] = joblib.load(model_path)

        # Carica lo scaler Y
        scaler_y_path = os.path.join(target_path, 'scaler_y.joblib')
        scalers_y[target] = joblib.load(scaler_y_path)

    print(f"Modelli e scaler caricati da: {base_path}")
    return models, scaler_X, scalers_y, target_variables




In [ ]:
save_models_and_scalers(models, scaler_X, scalers, target_variables)

In [ ]:
# Previsione delle variabili mancanti per data_before_2010
# Prepara data_before_2010
data_before_2010 = data_before_2010.sort_values('datetime')
data_before_2010.set_index('datetime', inplace=True)

# Assicurati che le features non abbiano valori mancanti
data_before_2010[features] = data_before_2010[features].ffill()
data_before_2010[features] = data_before_2010[features].bfill()

In [ ]:
models, scaler_X, scalers_y, target_variables = load_models_and_scalers()

timesteps = 24
# Crea X per data_before_2010
X_before = data_before_2010[features].values
X_before_scaled = scaler_X.transform(X_before)

# Crea le sequenze per LSTM
X_before_seq = create_sequences(timesteps, X_before_scaled)

# Prevedi le variabili mancanti
for i, target in enumerate(target_variables):
    print("Shape di X_before_seq:", X_before_seq.shape)
    print(f"Previsione di {target} per data_before_2010")
    y_pred_scaled = models[target].predict(X_before_seq)
    print("Shape delle predizioni:", y_pred_scaled.shape)
    # Ricostruisci i valori originali
    scaler = scalers_y[target]
    y_pred = scaler.inverse_transform(y_pred_scaled)

    # Allinea le previsioni con le date corrette
    dates = data_before_2010.index[timesteps:]
    data_before_2010.loc[dates, target] = y_pred

# Gestisci eventuali valori iniziali mancanti
data_before_2010[target_variables] = data_before_2010[target_variables].bfill()

# Combina data_before_2010 e data_after_2010
weather_data_complete = pd.concat([data_before_2010, data_after_2010], axis=0)
weather_data_complete = weather_data_complete.sort_index()

# Salva il dataset completo
weather_data_complete.reset_index(inplace=True)
weather_data_complete.to_parquet('/kaggle/working/weather_data_complete.parquet', index=False)


## 2. Esplorazione dei Dati Meteo

In [ ]:
weather_data = pd.read_parquet('/kaggle/working/weather_data_complete.parquet')

In [ ]:
# Visualizzazione delle tendenze temporali
fig, axes = plt.subplots(5, 1, figsize=(15, 20))
weather_data.set_index('date')['temp'].plot(ax=axes[0], title='Temperatura Media Giornaliera')
weather_data.set_index('date')['humidity'].plot(ax=axes[1], title='Umidità Media Giornaliera')
weather_data.set_index('date')['solarradiation'].plot(ax=axes[2], title='Radiazione Solare Giornaliera')
weather_data.set_index('date')['solarenergy'].plot(ax=axes[3], title='Radiazione Solare Giornaliera')
weather_data.set_index('date')['precip'].plot(ax=axes[4], title='Precipitazioni Giornaliere')
plt.tight_layout()
plt.show()
save_plot(plt, 'weather_trends', '/kaggle/working/plots')
plt.close()

## 3. Simulazione dei Dati di Produzione Annuale

In [ ]:

# Esempio di utilizzo
olive_varieties = pd.read_csv('/kaggle/input/olive-oil/variety_olive_oil_production.csv')


def add_olive_water_consumption_correlation(dataset):
    # Dati simulati per il fabbisogno d'acqua e la correlazione con la temperatura
    fabbisogno_acqua = {
        "Nocellara dell'Etna": {"Primavera": 1200, "Estate": 2000, "Autunno": 1000, "Inverno": 500, "Temperatura Ottimale": 18, "Resistenza": "Media"},
        "Leccino": {"Primavera": 1000, "Estate": 1800, "Autunno": 800, "Inverno": 400, "Temperatura Ottimale": 20, "Resistenza": "Alta"},
        "Frantoio": {"Primavera": 1100, "Estate": 1900, "Autunno": 900, "Inverno": 450, "Temperatura Ottimale": 19, "Resistenza": "Alta"},
        "Coratina": {"Primavera": 1300, "Estate": 2200, "Autunno": 1100, "Inverno": 550, "Temperatura Ottimale": 17, "Resistenza": "Media"},
        "Moraiolo": {"Primavera": 1150, "Estate": 2100, "Autunno": 900, "Inverno": 480, "Temperatura Ottimale": 18, "Resistenza": "Media"},
        "Pendolino": {"Primavera": 1050, "Estate": 1850, "Autunno": 850, "Inverno": 430, "Temperatura Ottimale": 20, "Resistenza": "Alta"},
        "Taggiasca": {"Primavera": 1000, "Estate": 1750, "Autunno": 800, "Inverno": 400, "Temperatura Ottimale": 19, "Resistenza": "Alta"},
        "Canino": {"Primavera": 1100, "Estate": 1900, "Autunno": 900, "Inverno": 450, "Temperatura Ottimale": 18, "Resistenza": "Media"},
        "Itrana": {"Primavera": 1200, "Estate": 2000, "Autunno": 1000, "Inverno": 500, "Temperatura Ottimale": 17, "Resistenza": "Media"},
        "Ogliarola": {"Primavera": 1150, "Estate": 1950, "Autunno": 900, "Inverno": 480, "Temperatura Ottimale": 18, "Resistenza": "Media"},
        "Biancolilla": {"Primavera": 1050, "Estate": 1800, "Autunno": 850, "Inverno": 430, "Temperatura Ottimale": 19, "Resistenza": "Alta"}
    }

    # Calcola il fabbisogno idrico annuale per ogni varietà
    for varieta in fabbisogno_acqua:
        fabbisogno_acqua[varieta]["Annuale"] = sum([fabbisogno_acqua[varieta][stagione] for stagione in ["Primavera", "Estate", "Autunno", "Inverno"]])

    # Aggiungiamo le nuove colonne al dataset
    dataset["Fabbisogno Acqua Primavera (m³/ettaro)"] = dataset["Varietà di Olive"].apply(lambda x: fabbisogno_acqua[x]["Primavera"])
    dataset["Fabbisogno Acqua Estate (m³/ettaro)"] = dataset["Varietà di Olive"].apply(lambda x: fabbisogno_acqua[x]["Estate"])
    dataset["Fabbisogno Acqua Autunno (m³/ettaro)"] = dataset["Varietà di Olive"].apply(lambda x: fabbisogno_acqua[x]["Autunno"])
    dataset["Fabbisogno Acqua Inverno (m³/ettaro)"] = dataset["Varietà di Olive"].apply(lambda x: fabbisogno_acqua[x]["Inverno"])
    dataset["Fabbisogno Idrico Annuale (m³/ettaro)"] = dataset["Varietà di Olive"].apply(lambda x: fabbisogno_acqua[x]["Annuale"])
    dataset["Temperatura Ottimale"] = dataset["Varietà di Olive"].apply(lambda x: fabbisogno_acqua[x]["Temperatura Ottimale"])
    dataset["Resistenza alla Siccità"] = dataset["Varietà di Olive"].apply(lambda x: fabbisogno_acqua[x]["Resistenza"])

    return dataset


olive_varieties = add_olive_water_consumption_correlation(olive_varieties)

olive_varieties.to_parquet("/kaggle/working/olive_varieties.parquet")

In [ ]:
def preprocess_weather_data(weather_df):
    # Calcola statistiche mensili per ogni anno
    monthly_weather = weather_df.groupby(['year', 'month']).agg({
        'temp': ['mean', 'min', 'max'],
        'humidity': 'mean',
        'precip': 'sum',
        'windspeed': 'mean',
        'cloudcover': 'mean',
        'solarradiation': 'sum',
        'solarenergy': 'sum',
        'uvindex': 'max'
    }).reset_index()

    monthly_weather.columns = ['year', 'month'] + [f'{col[0]}_{col[1]}' for col in monthly_weather.columns[2:]]
    return monthly_weather


def get_growth_phase(month):
    if month in [12, 1, 2]:
        return 'dormancy'
    elif month in [3, 4, 5]:
        return 'flowering'
    elif month in [6, 7, 8]:
        return 'fruit_set'
    else:
        return 'ripening'


def calculate_weather_effect(row, optimal_temp):
    # Effetti base
    temp_effect = -0.1 * (row['temp_mean'] - optimal_temp) ** 2
    rain_effect = -0.05 * (row['precip_sum'] - 600) ** 2 / 10000
    sun_effect = 0.1 * row['solarenergy_sum'] / 1000

    # Fattori di scala basati sulla fase di crescita
    if row['growth_phase'] == 'dormancy':
        temp_scale = 0.5
        rain_scale = 0.2
        sun_scale = 0.1
    elif row['growth_phase'] == 'flowering':
        temp_scale = 2.0
        rain_scale = 1.5
        sun_scale = 1.0
    elif row['growth_phase'] == 'fruit_set':
        temp_scale = 1.5
        rain_scale = 1.0
        sun_scale = 0.8
    else:  # ripening
        temp_scale = 1.0
        rain_scale = 0.5
        sun_scale = 1.2

    # Calcolo dell'effetto combinato
    combined_effect = (
            temp_scale * temp_effect +
            rain_scale * rain_effect +
            sun_scale * sun_effect
    )

    # Aggiustamenti specifici per fase
    if row['growth_phase'] == 'flowering':
        combined_effect -= 0.5 * max(0, row['precip_sum'] - 50)  # Penalità per pioggia eccessiva durante la fioritura
    elif row['growth_phase'] == 'fruit_set':
        combined_effect += 0.3 * max(0, row['temp_mean'] - (optimal_temp + 5))  # Bonus per temperature più alte durante la formazione dei frutti

    return combined_effect


def calculate_water_need(weather_data, base_need, optimal_temp):
    # Calcola il fabbisogno idrico basato su temperatura e precipitazioni
    temp_factor = 1 + 0.05 * (weather_data['temp_mean'] - optimal_temp)  # Aumenta del 5% per ogni grado sopra l'ottimale
    rain_factor = 1 - 0.001 * weather_data['precip_sum']  # Diminuisce leggermente con l'aumentare delle precipitazioni
    return base_need * temp_factor * rain_factor


def clean_column_name(name):
    # Rimuove caratteri speciali e spazi, converte in snake_case e abbrevia
    name = re.sub(r'[^a-zA-Z0-9\s]', '', name)  # Rimuove caratteri speciali
    name = name.lower().replace(' ', '_')  # Converte in snake_case

    # Abbreviazioni comuni
    abbreviations = {
        'production': 'prod',
        'percentage': 'pct',
        'hectare': 'ha',
        'tonnes': 't',
        'litres': 'l',
        'minimum': 'min',
        'maximum': 'max',
        'average': 'avg'
    }

    for full, abbr in abbreviations.items():
        name = name.replace(full, abbr)

    return name


def create_technique_mapping(olive_varieties, mapping_path='models/technique_mapping.joblib'):
    # Estrai tutte le tecniche uniche dal dataset e convertile in lowercase
    all_techniques = olive_varieties['Tecnica di Coltivazione'].str.lower().unique()

    # Crea il mapping partendo da 1
    technique_mapping = {tech: i + 1 for i, tech in enumerate(sorted(all_techniques))}

    # Salva il mapping
    os.makedirs(os.path.dirname(mapping_path), exist_ok=True)
    joblib.dump(technique_mapping, mapping_path)

    return technique_mapping


def encode_techniques(df, mapping_path='models/technique_mapping.joblib'):
    if not os.path.exists(mapping_path):
        raise FileNotFoundError(f"Mapping not found at {mapping_path}. Run create_technique_mapping first.")

    technique_mapping = joblib.load(mapping_path)

    # Trova tutte le colonne delle tecniche
    tech_columns = [col for col in df.columns if col.endswith('_tech')]

    # Applica il mapping a tutte le colonne delle tecniche
    for col in tech_columns:
        df[col] = df[col].str.lower().map(technique_mapping).fillna(0).astype(int)

    return df


def decode_techniques(df, mapping_path='models/technique_mapping.joblib'):
    if not os.path.exists(mapping_path):
        raise FileNotFoundError(f"Mapping not found at {mapping_path}")

    technique_mapping = joblib.load(mapping_path)
    reverse_mapping = {v: k for k, v in technique_mapping.items()}
    reverse_mapping[0] = ''  # Aggiungi un mapping per 0 a stringa vuota

    # Trova tutte le colonne delle tecniche
    tech_columns = [col for col in df.columns if col.endswith('_tech')]

    # Applica il reverse mapping a tutte le colonne delle tecniche
    for col in tech_columns:
        df[col] = df[col].map(reverse_mapping)

    return df


def decode_single_technique(technique_value, mapping_path='models/technique_mapping.joblib'):
    if not os.path.exists(mapping_path):
        raise FileNotFoundError(f"Mapping not found at {mapping_path}")

    technique_mapping = joblib.load(mapping_path)
    reverse_mapping = {v: k for k, v in technique_mapping.items()}
    reverse_mapping[0] = ''

    return reverse_mapping.get(technique_value, '')

In [ ]:
def simulate_olive_production(weather_data, olive_varieties, num_simulations=5, random_seed=None):
    """
    Simula la produzione di olive per diverse zone e varietà, considerando variazioni meteo specifiche per zona.
    Include barre di avanzamento per monitorare il progresso.
    
    Args:
        weather_data: DataFrame con dati meteorologici storici
        olive_varieties: DataFrame con informazioni sulle varietà di olive
        num_simulations: Numero di simulazioni/zone da generare
        random_seed: Seme per la riproducibilità dei risultati
    
    Returns:
        DataFrame con i risultati delle simulazioni per tutte le zone
    """
    if random_seed is not None:
        np.random.seed(random_seed)

    create_technique_mapping(olive_varieties)
    monthly_weather = preprocess_weather_data(weather_data)
    all_results = []

    # Preparazione dati varietà
    all_varieties = olive_varieties['Varietà di Olive'].unique()
    variety_techniques = {
        variety: olive_varieties[olive_varieties['Varietà di Olive'] == variety]['Tecnica di Coltivazione'].unique()
        for variety in all_varieties
    }

    # Barra di avanzamento principale per le simulazioni
    with tqdm(total=num_simulations*num_simulations, desc="Simulazioni completate") as sim_pbar:
        # Per ogni simulazione (anno)
        for sim in range(num_simulations):
            # Seleziona anno di base per questa simulazione
            selected_year = np.random.choice(monthly_weather['year'].unique())
            base_weather = monthly_weather[monthly_weather['year'] == selected_year].copy()
            base_weather.loc[:, 'growth_phase'] = base_weather['month'].apply(get_growth_phase)

            # Per ogni zona nella simulazione
            for zone in range(num_simulations):
                # Crea una copia dei dati meteo per questa zona specifica
                zone_weather = base_weather.copy()

                # Genera variazioni meteorologiche specifiche per questa zona
                zone_weather['temp_mean'] *= np.random.uniform(0.95, 1.05, len(zone_weather))
                zone_weather['precip_sum'] *= np.random.uniform(0.9, 1.1, len(zone_weather))
                zone_weather['solarenergy_sum'] *= np.random.uniform(0.95, 1.05, len(zone_weather))

                # Genera caratteristiche specifiche della zona
                num_varieties = np.random.randint(1, 4)  # 1-3 varietà per zona
                selected_varieties = np.random.choice(all_varieties, size=num_varieties, replace=False)
                hectares = np.random.uniform(1, 10)  # Dimensione del terreno
                percentages = np.random.dirichlet(np.ones(num_varieties))  # Distribuzione delle varietà

                # Inizializzazione contatori annuali
                annual_production = 0
                annual_min_oil = 0
                annual_max_oil = 0
                annual_avg_oil = 0
                annual_water_need = 0

                # Inizializzazione dizionario dati varietà
                variety_data = {clean_column_name(variety): {
                    'tech': '',
                    'pct': 0,
                    'prod_t_ha': 0,
                    'oil_prod_t_ha': 0,
                    'oil_prod_l_ha': 0,
                    'min_yield_pct': 0,
                    'max_yield_pct': 0,
                    'min_oil_prod_l_ha': 0,
                    'max_oil_prod_l_ha': 0,
                    'avg_oil_prod_l_ha': 0,
                    'l_per_t': 0,
                    'min_l_per_t': 0,
                    'max_l_per_t': 0,
                    'avg_l_per_t': 0,
                    'olive_prod': 0,
                    'min_oil_prod': 0,
                    'max_oil_prod': 0,
                    'avg_oil_prod': 0,
                    'water_need': 0
                } for variety in all_varieties}

                # Simula produzione per ogni varietà selezionata
                for i, variety in enumerate(selected_varieties):
                    # Seleziona tecnica di coltivazione casuale per questa varietà
                    technique = np.random.choice(variety_techniques[variety])
                    percentage = percentages[i]

                    # Ottieni informazioni specifiche della varietà
                    variety_info = olive_varieties[
                        (olive_varieties['Varietà di Olive'] == variety) &
                        (olive_varieties['Tecnica di Coltivazione'] == technique)
                        ].iloc[0]

                    # Calcola produzione base con variabilità
                    base_production = variety_info['Produzione (tonnellate/ettaro)'] * 1000 * percentage * hectares / 12
                    base_production *= np.random.uniform(0.9, 1.1)  # Aggiungi variabilità alla produzione base

                    # Calcola effetti meteo sulla produzione
                    weather_effect = zone_weather.apply(
                        lambda row: calculate_weather_effect(row, variety_info['Temperatura Ottimale']),
                        axis=1
                    )
                    monthly_production = base_production * (1 + weather_effect / 10000)
                    monthly_production *= np.random.uniform(0.95, 1.05, len(zone_weather))

                    # Calcola produzione annuale per questa varietà
                    annual_variety_production = monthly_production.sum()

                    # Calcola rese di olio con variabilità
                    min_yield_factor = np.random.uniform(0.95, 1.05)
                    max_yield_factor = np.random.uniform(0.95, 1.05)
                    avg_yield_factor = (min_yield_factor + max_yield_factor) / 2

                    min_oil_production = annual_variety_production * variety_info['Min Litri per Tonnellata'] / 1000 * min_yield_factor
                    max_oil_production = annual_variety_production * variety_info['Max Litri per Tonnellata'] / 1000 * max_yield_factor
                    avg_oil_production = annual_variety_production * variety_info['Media Litri per Tonnellata'] / 1000 * avg_yield_factor

                    # Calcola fabbisogno idrico
                    base_water_need = (
                                            variety_info['Fabbisogno Acqua Primavera (m³/ettaro)'] +
                                            variety_info['Fabbisogno Acqua Estate (m³/ettaro)'] +
                                            variety_info['Fabbisogno Acqua Autunno (m³/ettaro)'] +
                                            variety_info['Fabbisogno Acqua Inverno (m³/ettaro)']
                                    ) / 4  # Media stagionale

                    monthly_water_need = zone_weather.apply(
                        lambda row: calculate_water_need(row, base_water_need, variety_info['Temperatura Ottimale']),
                        axis=1
                    )
                    monthly_water_need *= np.random.uniform(0.95, 1.05, len(monthly_water_need))
                    annual_variety_water_need = monthly_water_need.sum() * percentage * hectares

                    # Aggiorna totali annuali
                    annual_production += annual_variety_production
                    annual_min_oil += min_oil_production
                    annual_max_oil += max_oil_production
                    annual_avg_oil += avg_oil_production
                    annual_water_need += annual_variety_water_need

                    # Aggiorna dati varietà
                    clean_variety = clean_column_name(variety)
                    variety_data[clean_variety].update({
                        'tech': clean_column_name(technique),
                        'pct': percentage,
                        'prod_t_ha': variety_info['Produzione (tonnellate/ettaro)'] * np.random.uniform(0.95, 1.05),
                        'oil_prod_t_ha': variety_info['Produzione Olio (tonnellate/ettaro)'] * np.random.uniform(0.95, 1.05),
                        'oil_prod_l_ha': variety_info['Produzione Olio (litri/ettaro)'] * np.random.uniform(0.95, 1.05),
                        'min_yield_pct': variety_info['Min % Resa'] * min_yield_factor,
                        'max_yield_pct': variety_info['Max % Resa'] * max_yield_factor,
                        'min_oil_prod_l_ha': variety_info['Min Produzione Olio (litri/ettaro)'] * min_yield_factor,
                        'max_oil_prod_l_ha': variety_info['Max Produzione Olio (litri/ettaro)'] * max_yield_factor,
                        'avg_oil_prod_l_ha': variety_info['Media Produzione Olio (litri/ettaro)'] * avg_yield_factor,
                        'l_per_t': variety_info['Litri per Tonnellata'] * np.random.uniform(0.98, 1.02),
                        'min_l_per_t': variety_info['Min Litri per Tonnellata'] * min_yield_factor,
                        'max_l_per_t': variety_info['Max Litri per Tonnellata'] * max_yield_factor,
                        'avg_l_per_t': variety_info['Media Litri per Tonnellata'] * avg_yield_factor,
                        'olive_prod': annual_variety_production,
                        'min_oil_prod': min_oil_production,
                        'max_oil_prod': max_oil_production,
                        'avg_oil_prod': avg_oil_production,
                        'water_need': annual_variety_water_need
                    })

                # Appiattisci i dati delle varietà per il DataFrame finale
                flattened_variety_data = {
                    f'{variety}_{key}': value
                    for variety, data in variety_data.items()
                    for key, value in data.items()
                }

                # Aggiungi il risultato con tutti i dati della zona
                all_results.append({
                    'simulation_id': sim + 1,
                    'zone_id': zone + 1,
                    'year': selected_year,
                    'temp_mean': zone_weather['temp_mean'].mean(),
                    'precip_sum': zone_weather['precip_sum'].sum(),
                    'solar_energy_sum': zone_weather['solarenergy_sum'].sum(),
                    'ha': hectares,
                    'zone': f"zone_{zone + 1}",
                    'olive_prod': annual_production,
                    'min_oil_prod': annual_min_oil,
                    'max_oil_prod': annual_max_oil,
                    'avg_oil_prod': annual_avg_oil,
                    'total_water_need': annual_water_need,
                    **flattened_variety_data
                })
                # Aggiorna la barra di avanzamento principale
                sim_pbar.update(1)
            

    # Crea DataFrame finale con tutti i risultati
    df_results = pd.DataFrame(all_results)
    return df_results


olive_varieties = pd.read_parquet("/kaggle/working/olive_varieties.parquet")

weather_data = pd.read_parquet('/kaggle/working/weather_data_complete.parquet')

simulated_data = simulate_olive_production(weather_data, olive_varieties, 1000, random_state_value)

simulated_data.to_parquet("/kaggle/working/simulated_data.parquet")


# Funzione per visualizzare il mapping delle tecniche
def print_technique_mapping(mapping_path='/kaggle/working/models/technique_mapping.joblib'):
    if not os.path.exists(mapping_path):
        print("Mapping file not found.")
        return

    mapping = joblib.load(mapping_path)
    print("Technique Mapping:")
    for technique, code in mapping.items():
        print(f"{technique}: {code}")


# Visualizza il mapping delle tecniche
print_technique_mapping()

In [ ]:
simulated_data = pd.read_parquet("/kaggle/working/simulated_data.parquet")


def clean_column_names(df):
    # Funzione per pulire i nomi delle colonne
    new_columns = []
    for col in df.columns:
        # Usa regex per separare le varietà
        varieties = re.findall(r'([a-z]+)_([a-z_]+)', col)
        if varieties:
            new_columns.append(f"{varieties[0][0]}_{varieties[0][1]}")
        else:
            new_columns.append(col)
    return new_columns


def prepare_comparison_data(simulated_data, olive_varieties):
    # Pulisci i nomi delle colonne
    df = simulated_data.copy()

    df.columns = clean_column_names(df)
    df = encode_techniques(df)

    all_varieties = olive_varieties['Varietà di Olive'].unique()
    varieties = [clean_column_name(variety) for variety in all_varieties]
    comparison_data = []

    for variety in varieties:
        olive_prod_col = next((col for col in df.columns if col.startswith(f'{variety}_') and col.endswith('_olive_prod')), None)
        oil_prod_col = next((col for col in df.columns if col.startswith(f'{variety}_') and col.endswith('_avg_oil_prod')), None)
        tech_col = next((col for col in df.columns if col.startswith(f'{variety}_') and col.endswith('_tech')), None)
        water_need_col = next((col for col in df.columns if col.startswith(f'{variety}_') and col.endswith('_water_need')), None)

        if olive_prod_col and oil_prod_col and tech_col and water_need_col:
            variety_data = df[[olive_prod_col, oil_prod_col, tech_col, water_need_col]]
            variety_data = variety_data[variety_data[tech_col] != 0]  # Esclude le righe dove la tecnica è 0

            if not variety_data.empty:
                avg_olive_prod = pd.to_numeric(variety_data[olive_prod_col], errors='coerce').mean()
                avg_oil_prod = pd.to_numeric(variety_data[oil_prod_col], errors='coerce').mean()
                avg_water_need = pd.to_numeric(variety_data[water_need_col], errors='coerce').mean()
                efficiency = avg_oil_prod / avg_olive_prod if avg_olive_prod > 0 else 0
                water_efficiency = avg_oil_prod / avg_water_need if avg_water_need > 0 else 0

                comparison_data.append({
                    'Variety': variety,
                    'Avg Olive Production (kg/ha)': avg_olive_prod,
                    'Avg Oil Production (L/ha)': avg_oil_prod,
                    'Avg Water Need (m³/ha)': avg_water_need,
                    'Oil Efficiency (L/kg)': efficiency,
                    'Water Efficiency (L oil/m³ water)': water_efficiency
                })

    return pd.DataFrame(comparison_data)


def plot_variety_comparison(comparison_data, metric):
    plt.figure(figsize=(12, 6))
    bars = plt.bar(comparison_data['Variety'], comparison_data[metric])
    plt.title(f'Comparison of {metric} across Olive Varieties')
    plt.xlabel('Variety')
    plt.ylabel(metric)
    plt.xticks(rotation=45, ha='right')

    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width() / 2., height,
                 f'{height:.2f}',
                 ha='center', va='bottom')

    plt.tight_layout()
    plt.show()
    save_plot(plt, f'variety_comparison_{metric.lower().replace(" ", "_").replace("/", "_").replace("(", "").replace(")", "")}', '/kaggle/working/plots')
    plt.close()


def plot_efficiency_vs_production(comparison_data):
    plt.figure(figsize=(10, 6))

    plt.scatter(comparison_data['Avg Olive Production (kg/ha)'],
                comparison_data['Oil Efficiency (L/kg)'],
                s=100)

    for i, row in comparison_data.iterrows():
        plt.annotate(row['Variety'],
                     (row['Avg Olive Production (kg/ha)'], row['Oil Efficiency (L/kg)']),
                     xytext=(5, 5), textcoords='offset points')

    plt.title('Oil Efficiency vs Olive Production by Variety')
    plt.xlabel('Average Olive Production (kg/ha)')
    plt.ylabel('Oil Efficiency (L oil / kg olives)')
    plt.tight_layout()
    save_plot(plt, 'efficiency_vs_production', '/kaggle/working/plots')
    plt.close()


def plot_water_efficiency_vs_production(comparison_data):
    plt.figure(figsize=(10, 6))

    plt.scatter(comparison_data['Avg Olive Production (kg/ha)'],
                comparison_data['Water Efficiency (L oil/m³ water)'],
                s=100)

    for i, row in comparison_data.iterrows():
        plt.annotate(row['Variety'],
                     (row['Avg Olive Production (kg/ha)'], row['Water Efficiency (L oil/m³ water)']),
                     xytext=(5, 5), textcoords='offset points')

    plt.title('Water Efficiency vs Olive Production by Variety')
    plt.xlabel('Average Olive Production (kg/ha)')
    plt.ylabel('Water Efficiency (L oil / m³ water)')
    plt.tight_layout()
    plt.show()
    save_plot(plt, 'water_efficiency_vs_production', '/kaggle/working/plots')
    plt.close()


def plot_water_need_vs_oil_production(comparison_data):
    plt.figure(figsize=(10, 6))

    plt.scatter(comparison_data['Avg Water Need (m³/ha)'],
                comparison_data['Avg Oil Production (L/ha)'],
                s=100)

    for i, row in comparison_data.iterrows():
        plt.annotate(row['Variety'],
                     (row['Avg Water Need (m³/ha)'], row['Avg Oil Production (L/ha)']),
                     xytext=(5, 5), textcoords='offset points')

    plt.title('Oil Production vs Water Need by Variety')
    plt.xlabel('Average Water Need (m³/ha)')
    plt.ylabel('Average Oil Production (L/ha)')
    plt.tight_layout()
    plt.show()
    save_plot(plt, 'water_need_vs_oil_production', '/kaggle/working/plots')
    plt.close()


def analyze_by_technique(simulated_data, olive_varieties):
    # Pulisci i nomi delle colonne
    df = simulated_data.copy()

    df.columns = clean_column_names(df)
    df = encode_techniques(df)
    all_varieties = olive_varieties['Varietà di Olive'].unique()
    varieties = [clean_column_name(variety) for variety in all_varieties]

    technique_data = []

    for variety in varieties:
        olive_prod_col = next((col for col in df.columns if col.startswith(f'{variety}_') and col.endswith('_olive_prod')), None)
        oil_prod_col = next((col for col in df.columns if col.startswith(f'{variety}_') and col.endswith('_avg_oil_prod')), None)
        tech_col = next((col for col in df.columns if col.startswith(f'{variety}_') and col.endswith('_tech')), None)
        water_need_col = next((col for col in df.columns if col.startswith(f'{variety}_') and col.endswith('_water_need')), None)

        if olive_prod_col and oil_prod_col and tech_col and water_need_col:
            variety_data = df[[olive_prod_col, oil_prod_col, tech_col, water_need_col]]
            variety_data = variety_data[variety_data[tech_col] != 0]

            if not variety_data.empty:
                for tech in variety_data[tech_col].unique():
                    tech_data = variety_data[variety_data[tech_col] == tech]

                    avg_olive_prod = pd.to_numeric(tech_data[olive_prod_col], errors='coerce').mean()
                    avg_oil_prod = pd.to_numeric(tech_data[oil_prod_col], errors='coerce').mean()
                    avg_water_need = pd.to_numeric(tech_data[water_need_col], errors='coerce').mean()

                    efficiency = avg_oil_prod / avg_olive_prod if avg_olive_prod > 0 else 0
                    water_efficiency = avg_oil_prod / avg_water_need if avg_water_need > 0 else 0

                    technique_data.append({
                        'Variety': variety,
                        'Technique': tech,
                        'Technique String': decode_single_technique(tech),
                        'Avg Olive Production (kg/ha)': avg_olive_prod,
                        'Avg Oil Production (L/ha)': avg_oil_prod,
                        'Avg Water Need (m³/ha)': avg_water_need,
                        'Oil Efficiency (L/kg)': efficiency,
                        'Water Efficiency (L oil/m³ water)': water_efficiency
                    })

    return pd.DataFrame(technique_data)


# Esecuzione dell'analisi
comparison_data = prepare_comparison_data(simulated_data, olive_varieties)

# Genera i grafici
plot_variety_comparison(comparison_data, 'Avg Olive Production (kg/ha)')
plot_variety_comparison(comparison_data, 'Avg Oil Production (L/ha)')
plot_variety_comparison(comparison_data, 'Avg Water Need (m³/ha)')
plot_variety_comparison(comparison_data, 'Oil Efficiency (L/kg)')
plot_variety_comparison(comparison_data, 'Water Efficiency (L oil/m³ water)')
plot_efficiency_vs_production(comparison_data)
plot_water_efficiency_vs_production(comparison_data)
plot_water_need_vs_oil_production(comparison_data)

# Analisi per tecnica
technique_data = analyze_by_technique(simulated_data, olive_varieties)

print(technique_data)

# Stampa un sommario statistico
print("Comparison by Variety:")
print(comparison_data.set_index('Variety'))
print("\nBest Varieties by Water Efficiency:")
print(comparison_data.sort_values('Water Efficiency (L oil/m³ water)', ascending=False).head())

## 4. Analisi della Relazione tra Meteo e Produzione

In [ ]:
def get_full_data(simulated_data, olive_varieties):
    # Assumiamo che simulated_data contenga già tutti i dati necessari
    # Includiamo solo le colonne rilevanti
    relevant_columns = ['year', 'temp_mean', 'precip_sum', 'solar_energy_sum', 'ha', 'zone', 'olive_prod']

    # Aggiungiamo le colonne specifiche per varietà
    all_varieties = olive_varieties['Varietà di Olive'].unique()
    varieties = [clean_column_name(variety) for variety in all_varieties]
    for variety in varieties:
        relevant_columns.extend([f'{variety}_olive_prod', f'{variety}_tech'])

    return simulated_data[relevant_columns].copy()


def analyze_correlations(full_data, variety):
    # Filtra i dati per la varietà specifica
    variety_data = full_data[[col for col in full_data.columns if not col.startswith('_') or col.startswith(f'{variety}_')]]

    # Rinomina le colonne per chiarezza
    variety_data = variety_data.rename(columns={
        f'{variety}_olive_prod': 'olive_production',
        f'{variety}_tech': 'technique'
    })

    # Matrice di correlazione
    plt.figure(figsize=(12, 10))
    corr_matrix = variety_data[['temp_mean', 'precip_sum', 'solar_energy_sum', 'olive_production']].corr()
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
    plt.title(f'Matrice di Correlazione - {variety}')
    plt.tight_layout()
    plt.show()
    save_plot(plt, f'correlation_matrix_{variety}', '/kaggle/working/plots')
    plt.close()

    # Scatter plots
    fig, axes = plt.subplots(2, 2, figsize=(20, 20))
    fig.suptitle(f'Relazione tra Fattori Meteorologici e Produzione di Olive - {variety}', fontsize=16)

    for ax, var in zip(axes.flat, ['temp_mean', 'precip_sum', 'solar_energy_sum', 'ha']):
        sns.scatterplot(data=variety_data, x=var, y='olive_production', hue='technique', ax=ax)
        ax.set_title(f'{var.capitalize()} vs Produzione Olive')
        ax.set_xlabel(var.capitalize())
        ax.set_ylabel('Produzione Olive (kg/ettaro)')

    plt.tight_layout()
    plt.show()
    save_plot(plt, f'meteorological_factors_{variety}', '/kaggle/working/plots')
    plt.close()


# Uso delle funzioni
full_data = get_full_data(simulated_data, olive_varieties)

# Assumiamo che 'selected_variety' sia definito altrove nel codice
# Per esempio:
selected_variety = 'nocellara_delletna'

analyze_correlations(full_data, selected_variety)

## 5. Preparazione del Modello di Machine Learning

In [ ]:
def prepare_data(df, olive_varieties_df):
    # Crea una copia del DataFrame per evitare modifiche all'originale
    df = df.copy()

    # Ordina per zona e anno
    df = df.sort_values(['zone', 'year'])

    # Definisci le feature
    temporal_features = ['temp_mean', 'precip_sum', 'solar_energy_sum']
    static_features = ['ha']  # Feature statiche base
    target_features = ['olive_prod', 'min_oil_prod', 'max_oil_prod', 'avg_oil_prod', 'total_water_need']

    # Ottieni le varietà pulite
    varieties = [clean_column_name(variety) for variety in olive_varieties_df['Varietà di Olive']]

    # Crea la struttura delle feature per ogni varietà
    variety_features = [
        'tech', 'pct', 'prod_t_ha', 'oil_prod_t_ha', 'oil_prod_l_ha',
        'min_yield_pct', 'max_yield_pct', 'min_oil_prod_l_ha', 'max_oil_prod_l_ha',
        'avg_oil_prod_l_ha', 'l_per_t', 'min_l_per_t', 'max_l_per_t', 'avg_l_per_t'
    ]

    # Prepara dizionari per le nuove colonne
    new_columns = {}

    # Prepara le feature per ogni varietà
    for variety in varieties:
        # Feature esistenti
        for feature in variety_features:
            col_name = f"{variety}_{feature}"
            if col_name in df.columns:
                if feature != 'tech':  # Non includere la colonna tech direttamente
                    static_features.append(col_name)

        # Feature binarie per le tecniche di coltivazione
        for technique in ['tradizionale', 'intensiva', 'superintensiva']:
            col_name = f"{variety}_{technique}"
            new_columns[col_name] = df[f"{variety}_tech"].notna() & (
                    df[f"{variety}_tech"].str.lower() == technique
            ).fillna(False)
            static_features.append(col_name)

    # Aggiungi tutte le nuove colonne in una volta sola
    new_df = pd.concat([df] + [pd.Series(v, name=k) for k, v in new_columns.items()], axis=1)

    # Ordiniamo per zona e anno per mantenere la continuità temporale
    df_sorted = new_df.sort_values(['zone', 'year'])

    # Definiamo la dimensione della finestra temporale
    window_size = 41

    # Liste per raccogliere i dati
    temporal_sequences = []
    static_features_list = []
    targets_list = []

    # Iteriamo per ogni zona
    for zone in df_sorted['zone'].unique():
        zone_data = df_sorted[df_sorted['zone'] == zone].reset_index(drop=True)

        if len(zone_data) >= window_size:  # Verifichiamo che ci siano abbastanza dati
            # Creiamo sequenze temporali scorrevoli
            for i in range(len(zone_data) - window_size + 1):
                # Sequenza temporale
                temporal_window = zone_data.iloc[i:i + window_size][temporal_features].values
                # Verifichiamo che non ci siano valori NaN
                if not np.isnan(temporal_window).any():
                    temporal_sequences.append(temporal_window)

                    # Feature statiche (prendiamo quelle dell'ultimo timestep della finestra)
                    static_features_list.append(zone_data.iloc[i + window_size - 1][static_features].values)

                    # Target (prendiamo quelli dell'ultimo timestep della finestra)
                    targets_list.append(zone_data.iloc[i + window_size - 1][target_features].values)

    # Convertiamo in array numpy
    X_temporal = np.array(temporal_sequences)
    X_static = np.array(static_features_list)
    y = np.array(targets_list)

    print(f"Dataset completo - Temporal: {X_temporal.shape}, Static: {X_static.shape}, Target: {y.shape}")

    # Split dei dati (usando indici casuali per una migliore distribuzione)
    indices = np.random.permutation(len(X_temporal))
    #train_idx = int(len(indices) * 0.7)
    #val_idx = int(len(indices) * 0.85)
    
    train_idx = int(len(indices) * 0.65)    # 65% training
    val_idx = int(len(indices) * 0.85)      # 20% validation
    # Il resto rimane 15% test

    # Oppure versione con 25% validation:
    #train_idx = int(len(indices) * 0.60)    # 60% training
    #val_idx = int(len(indices) * 0.85)      # 25% validation

    train_indices = indices[:train_idx]
    val_indices = indices[train_idx:val_idx]
    test_indices = indices[val_idx:]

    # Split dei dati
    X_temporal_train = X_temporal[train_indices]
    X_temporal_val = X_temporal[val_indices]
    X_temporal_test = X_temporal[test_indices]

    X_static_train = X_static[train_indices]
    X_static_val = X_static[val_indices]
    X_static_test = X_static[test_indices]

    y_train = y[train_indices]
    y_val = y[val_indices]
    y_test = y[test_indices]

    # Standardizzazione
    scaler_temporal = StandardScaler()
    scaler_static = StandardScaler()
    scaler_y = StandardScaler()

    # Standardizzazione dei dati temporali
    X_temporal_train = scaler_temporal.fit_transform(X_temporal_train.reshape(-1, len(temporal_features))).reshape(X_temporal_train.shape)
    X_temporal_val = scaler_temporal.transform(X_temporal_val.reshape(-1, len(temporal_features))).reshape(X_temporal_val.shape)
    X_temporal_test = scaler_temporal.transform(X_temporal_test.reshape(-1, len(temporal_features))).reshape(X_temporal_test.shape)

    # Standardizzazione dei dati statici
    X_static_train = scaler_static.fit_transform(X_static_train)
    X_static_val = scaler_static.transform(X_static_val)
    X_static_test = scaler_static.transform(X_static_test)

    # Standardizzazione dei target
    y_train = scaler_y.fit_transform(y_train)
    y_val = scaler_y.transform(y_val)
    y_test = scaler_y.transform(y_test)

    print("\nShape dopo lo split e standardizzazione:")
    print(f"Train - Temporal: {X_temporal_train.shape}, Static: {X_static_train.shape}, Target: {y_train.shape}")
    print(f"Val - Temporal: {X_temporal_val.shape}, Static: {X_static_val.shape}, Target: {y_val.shape}")
    print(f"Test - Temporal: {X_temporal_test.shape}, Static: {X_static_test.shape}, Target: {y_test.shape}")

    # Prepara i dizionari di input
    train_data = {'temporal': X_temporal_train, 'static': X_static_train}
    val_data = {'temporal': X_temporal_val, 'static': X_static_val}
    test_data = {'temporal': X_temporal_test, 'static': X_static_test}

    return (train_data, y_train), (val_data, y_val), (test_data, y_test), (scaler_temporal, scaler_static, scaler_y)

## Divisione train/validation/test:


In [ ]:
simulated_data = pd.read_parquet("/kaggle/working/simulated_data.parquet")
olive_varieties = pd.read_parquet("/kaggle/working/olive_varieties.parquet")

(train_data, train_targets), (val_data, val_targets), (test_data, test_targets), scalers = prepare_data(simulated_data, olive_varieties)

scaler_temporal, scaler_static, scaler_y = scalers

print("Temporal data shape:", train_data['temporal'].shape)
print("Static data shape:", train_data['static'].shape)
print("Target shape:", train_targets.shape)

## OliveOilTransformer

In [ ]:
import tensorflow as tf
import numpy as np

class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, position, d_model):
        super(PositionalEncoding, self).__init__()
        self.pos_encoding = self.positional_encoding(position, d_model)

    def get_angles(self, position, i, d_model):
        angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
        return position * angles

    def positional_encoding(self, position, d_model):
        angle_rads = self.get_angles(
            position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
            i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
            d_model=d_model)

        sines = tf.math.sin(angle_rads[:, 0::2])
        cosines = tf.math.cos(angle_rads[:, 1::2])

        pos_encoding = tf.concat([sines, cosines], axis=-1)
        pos_encoding = pos_encoding[tf.newaxis, ...]
        return tf.cast(pos_encoding, tf.float32)

    def call(self, inputs):
        return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

class TemporalAugmentation(tf.keras.layers.Layer):
    def __init__(self, noise_factor=0.03, **kwargs):
        super().__init__(**kwargs)
        self.noise_factor = noise_factor

    def call(self, inputs, training=None):
        if training:
            noise = tf.random.normal(
                shape=tf.shape(inputs), 
                mean=0.0, 
                stddev=self.noise_factor
            )
            return inputs + noise
        return inputs

class EnhancedTransformerBlock(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, ff_dim, dropout=0.1):
        super().__init__()
        self.att = tf.keras.layers.MultiHeadAttention(
            num_heads=num_heads, 
            key_dim=d_model // num_heads,
            value_dim=d_model // num_heads
        )
        self.ffn = tf.keras.Sequential([
            tf.keras.layers.Dense(ff_dim, activation="gelu"),
            tf.keras.layers.Dropout(dropout),
            tf.keras.layers.Dense(d_model)
        ])
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = tf.keras.layers.Dropout(dropout)
        self.dropout2 = tf.keras.layers.Dropout(dropout)
        self.residual_attention = tf.keras.layers.Dense(d_model, activation='sigmoid')

    def call(self, inputs, training):
        # Self-attention con residual connection
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        residual_weights = self.residual_attention(inputs)
        out1 = self.layernorm1(inputs + residual_weights * attn_output)
        
        # Feed-forward con residual connection
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

class TemporalPoolingLayer(tf.keras.layers.Layer):
    def __init__(self, num_heads, key_dim, **kwargs):
        super().__init__(**kwargs)
        self.attention_pooling = tf.keras.layers.MultiHeadAttention(
            num_heads=num_heads, 
            key_dim=key_dim
        )
        self.temporal_pooling = tf.keras.layers.GlobalAveragePooling1D()
        self.max_pooling = tf.keras.layers.GlobalMaxPooling1D()
        self.concat = tf.keras.layers.Concatenate(axis=-1)
        
    def call(self, inputs, training=None):
        # Attention pooling
        att_output = self.attention_pooling(inputs, inputs)
        
        # Global average e max pooling
        avg_output = self.temporal_pooling(inputs)
        max_output = self.max_pooling(inputs)
        
        # Reshape attention output
        att_output = tf.reduce_mean(att_output, axis=1)
        
        # Concatena tutti i tipi di pooling
        return self.concat([att_output, avg_output, max_output])

class OliveOilTransformer(tf.keras.Model):
    def __init__(self, temporal_shape, static_shape, num_outputs,
                 d_model=128, num_heads=8, ff_dim=256, num_transformer_blocks=4,
                 mlp_units=[256, 128, 64], dropout=0.2):
        super(OliveOilTransformer, self).__init__()
        
        # Input layers
        self.temporal_input = tf.keras.layers.Input(shape=temporal_shape, name='temporal_input')
        self.static_input = tf.keras.layers.Input(shape=static_shape, name='static_input')
        
        # Input normalization
        self.temporal_normalization = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.static_normalization = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        
        # Data Augmentation
        self.temporal_augmentation = TemporalAugmentation(noise_factor=0.03)
        
        # Temporal path
        self.temporal_projection = tf.keras.Sequential([
            tf.keras.layers.Dense(d_model//2, activation='gelu',
                                kernel_regularizer=tf.keras.regularizers.l2(1e-5)),
            tf.keras.layers.Dropout(dropout),
            tf.keras.layers.Dense(d_model, activation='gelu',
                                kernel_regularizer=tf.keras.regularizers.l2(1e-5))
        ])
        
        self.pos_encoding = PositionalEncoding(position=temporal_shape[0], d_model=d_model)
        
        # Transformer blocks
        self.transformer_blocks = [
            EnhancedTransformerBlock(d_model, num_heads, ff_dim, dropout)
            for _ in range(num_transformer_blocks)
        ]
        
        # Temporal pooling
        self.temporal_pooling = TemporalPoolingLayer(
            num_heads=num_heads,
            key_dim=d_model//4
        )
        
        # Static path
        self.static_encoder = tf.keras.Sequential([
            tf.keras.layers.Dense(256, activation='gelu',
                                kernel_regularizer=tf.keras.regularizers.l2(1e-5)),
            tf.keras.layers.Dropout(dropout),
            tf.keras.layers.Dense(128, activation='gelu',
                                kernel_regularizer=tf.keras.regularizers.l2(1e-5)),
            tf.keras.layers.Dropout(dropout),
            tf.keras.layers.Dense(64, activation='gelu',
                                kernel_regularizer=tf.keras.regularizers.l2(1e-5))
        ])
        
        # Feature fusion
        self.fusion_layer = tf.keras.layers.Concatenate()
        
        # MLP head
        self.mlp_layers = []
        for units in mlp_units:
            self.mlp_layers.extend([
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.Dense(units, activation="gelu",
                                    kernel_regularizer=tf.keras.regularizers.l2(1e-5)),
                tf.keras.layers.Dropout(dropout)
            ])
        
        # Output layer
        self.final_layer = tf.keras.layers.Dense(
            num_outputs,
            activation='linear',
            kernel_regularizer=tf.keras.regularizers.l2(1e-5)
        )
        
        # Build model
        temporal_encoded = self.encode_temporal(self.temporal_input, training=True)
        static_encoded = self.encode_static(self.static_input)
        combined = self.fusion_layer([temporal_encoded, static_encoded])
        
        x = combined
        for layer in self.mlp_layers:
            x = layer(x)
            
        outputs = self.final_layer(x)
        
        self._model = tf.keras.Model(
            inputs={'temporal': self.temporal_input, 'static': self.static_input},
            outputs=outputs
        )

    def encode_temporal(self, x, training=None):
        # Normalization e augmentation
        x = self.temporal_normalization(x)
        x = self.temporal_augmentation(x, training=training)
        
        # Projection e positional encoding
        x = self.temporal_projection(x)
        x = self.pos_encoding(x)
        
        # Transformer blocks
        skip_connection = x
        for transformer in self.transformer_blocks:
            x = transformer(x, training=training)
        x = tf.keras.layers.Add()([x, skip_connection])
        
        # Pooling
        return self.temporal_pooling(x)

    def encode_static(self, x):
        x = self.static_normalization(x)
        return self.static_encoder(x)

    def call(self, inputs, training=None):
        temporal_input = inputs['temporal']
        static_input = inputs['static']
        
        temporal_encoded = self.encode_temporal(temporal_input, training)
        static_encoded = self.encode_static(static_input)
        
        combined = self.fusion_layer([temporal_encoded, static_encoded])
        
        x = combined
        for layer in self.mlp_layers:
            x = layer(x, training=training)
            
        return self.final_layer(x)

    def model(self):
        return self._model


# Configurazione del modello e training
def create_and_compile_model(temporal_shape, static_shape, num_outputs):
    model = OliveOilTransformer(
        temporal_shape=temporal_shape,
        static_shape=static_shape,
        num_outputs=num_outputs,
        d_model=128,
        num_heads=8,
        ff_dim=256,
        num_transformer_blocks=4,
        mlp_units=[256, 128, 64],
        dropout=0.2
    )

    class WarmUpLearningRateSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
        def __init__(self, initial_learning_rate=1e-3, warmup_steps=1000, decay_steps=10000):
            super().__init__()
            self.initial_learning_rate = initial_learning_rate
            self.warmup_steps = warmup_steps
            self.decay_steps = decay_steps

        def __call__(self, step):
            warmup_pct = tf.cast(step, tf.float32) / self.warmup_steps
            warmup_lr = self.initial_learning_rate * warmup_pct

            decay_factor = tf.pow(0.1, tf.cast(step, tf.float32) / self.decay_steps)
            decayed_lr = self.initial_learning_rate * decay_factor

            final_lr = tf.where(step < self.warmup_steps, warmup_lr, decayed_lr)
            return final_lr

        def get_config(self):
            return {
                'initial_learning_rate': self.initial_learning_rate,
                'warmup_steps': self.warmup_steps,
                'decay_steps': self.decay_steps
            }

    # Custom Metric per target
    class TargetSpecificMetric(tf.keras.callbacks.Callback):
        def __init__(self, validation_data, target_names):
            super().__init__()
            self.validation_data = validation_data
            self.target_names = target_names

        def on_epoch_end(self, epoch, logs={}):
            x_val, y_val = self.validation_data
            y_pred = self.model.predict(x_val)

            for i, name in enumerate(self.target_names):
                # Calcola MAE usando numpy
                mae = np.mean(np.abs(y_val[:, i] - y_pred[:, i]))
                logs[f'val_{name}_mae'] = mae

    # Target names per il monitoraggio specifico
    target_names = ['olive_prod', 'min_oil_prod', 'max_oil_prod', 
                    'avg_oil_prod', 'total_water_need']

    # Learning rate schedule
    lr_schedule = WarmUpLearningRateSchedule(
        initial_learning_rate=1e-3,
        warmup_steps=500,
        decay_steps=5000
    )



    # Callbacks setup
    callbacks = [
        # Early Stopping migliorato
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=20,
            restore_best_weights=True,
            min_delta=0.0005,
            mode='min'
        ),

        # Model Checkpoint
        tf.keras.callbacks.ModelCheckpoint(
            filepath='/kaggle/working/transformer/best_model_{epoch:02d}_{val_loss:.4f}.keras',
            monitor='val_loss',
            save_best_only=True,
            mode='min'
        ),

        # Metric per target specifici
        TargetSpecificMetric(
            validation_data=(val_data, val_targets),
            target_names=target_names
        ),

        # TensorBoard logging
        tf.keras.callbacks.TensorBoard(
            log_dir='/kaggle/working/transformer/logs',
            histogram_freq=1,
            write_graph=True,
            update_freq='epoch'
        )
    ]

    # Ricompila il modello con il nuovo optimizer
    model.compile(
        optimizer=tf.keras.optimizers.AdamW(
            learning_rate=lr_schedule,
            weight_decay=0.01
        ),
        loss=tf.keras.losses.Huber(),
        metrics=['mae']
    )

    return model, callbacks


# Creazione e compilazione del modello
model, callbacks = create_and_compile_model(
    temporal_shape=(train_data['temporal'].shape[1], train_data['temporal'].shape[2]),
    static_shape=(train_data['static'].shape[1],),
    num_outputs=train_targets.shape[1]
)

# Mostra il summary
model.model().summary()

#tf.keras.utils.plot_model(model.model, "/kaggle/working/models/oil_transformer_model.png", show_shapes=True, show_layer_names=True)

## Model Training

In [ ]:
# Esegui il training
history = model.fit(
    x=train_data,
    y=train_targets,
    validation_data=(val_data, val_targets),
    epochs=150,
    batch_size=64,
    callbacks=callbacks,
    verbose=1,
    shuffle=True
)

In [ ]:
# Per denormalizzare e calcolare l'errore reale
def calculate_real_error(model, test_data, test_targets, scaler_y):
    # Fare predizioni
    predictions = model.predict(test_data)
    
    # Denormalizzare predizioni e target
    predictions_real = scaler_y.inverse_transform(predictions)
    targets_real = scaler_y.inverse_transform(test_targets)
    
    # Calcolare errore percentuale per ogni target
    percentage_errors = []
    absolute_errors = []
    
    for i in range(predictions_real.shape[1]):
        mae = np.mean(np.abs(predictions_real[:, i] - targets_real[:, i]))
        mape = np.mean(np.abs((predictions_real[:, i] - targets_real[:, i]) / targets_real[:, i])) * 100
        percentage_errors.append(mape)
        absolute_errors.append(mae)
        
    # Stampa risultati per ogni target
    target_names = ['olive_prod', 'min_oil_prod', 'max_oil_prod', 'avg_oil_prod', 'total_water_need']
    
    print("\nErrori per target:")
    print("-" * 50)
    for i, target in enumerate(target_names):
        print(f"{target}:")
        print(f"MAE assoluto: {absolute_errors[i]:.2f}")
        print(f"Errore percentuale medio: {percentage_errors[i]:.2f}%")
        print(f"Precisione: {100 - percentage_errors[i]:.2f}%")
        print("-" * 50)
        
    return percentage_errors, absolute_errors

# Calcola gli errori reali
percentage_errors, absolute_errors = calculate_real_error(model, val_data, val_targets, scaler_y)

In [ ]:
def save_transformer_model(model, scaler_temporal, scaler_static, scaler_y, base_path='/kaggle/working/models/oli_transformer'):
    """
    Salva il modello transformer e i suoi scaler.
    
    Parameters:
    -----------
    model : OliveOilTransformer
        Il modello transformer
    scaler_temporal : StandardScaler
        Scaler per i dati temporali
    scaler_static : StandardScaler
        Scaler per i dati statici
    scaler_y : StandardScaler
        Scaler per i target
    base_path : str
        Percorso base dove salvare il modello e gli scaler
    """
    # Crea la cartella se non esiste
    os.makedirs(base_path, exist_ok=True)
    
    # Salva il modello
    model_path = os.path.join(base_path, 'olive_transformer.keras')
    model.save(model_path)
    
    # Salva gli scaler
    joblib.dump(scaler_temporal, os.path.join(base_path, 'scaler_temporal.joblib'))
    joblib.dump(scaler_static, os.path.join(base_path, 'scaler_static.joblib'))
    joblib.dump(scaler_y, os.path.join(base_path, 'scaler_y.joblib'))
    
    print(f"Modello transformer e scaler salvati in: {base_path}")

def load_transformer_model(base_path='/kaggle/working/models/oli_transformer'):
    """
    Carica il modello transformer e i suoi scaler.
    
    Parameters:
    -----------
    base_path : str
        Percorso dove sono salvati il modello e gli scaler
        
    Returns:
    --------
    tuple
        (model, scaler_temporal, scaler_static, scaler_y)
    """
    # Carica il modello
    model_path = os.path.join(base_path, 'olive_transformer.keras')
    model = tf.keras.models.load_model(model_path, custom_objects={
        'WarmUpLearningRateSchedule': WarmUpLearningRateSchedule
    })
    
    # Carica gli scaler
    scaler_temporal = joblib.load(os.path.join(base_path, 'scaler_temporal.joblib'))
    scaler_static = joblib.load(os.path.join(base_path, 'scaler_static.joblib'))
    scaler_y = joblib.load(os.path.join(base_path, 'scaler_y.joblib'))
    
    print(f"Modello transformer e scaler caricati da: {base_path}")
    return model, scaler_temporal, scaler_static, scaler_y

# Esempio di utilizzo:

# Per salvare:
save_transformer_model(
    model=model,
    scaler_temporal=scaler_temporal,
    scaler_static=scaler_static,
    scaler_y=scaler_y,
)

# Per caricare:
#model, scaler_temporal, scaler_static, scaler_y = load_transformer_model()


## 8. Conclusioni e Prossimi Passi

In questo notebook, abbiamo:
1. Caricato e analizzato i dati meteorologici
2. Simulato la produzione annuale di olive basata sui dati meteo
3. Esplorato le relazioni tra variabili meteorologiche e produzione di olive
4. Creato e valutato un modello di machine learning per prevedere la produzione
5. Utilizzato ARIMA per fare previsioni meteo
6. Previsto la produzione di olive per il prossimo anno

Prossimi passi:
- Raccogliere dati reali sulla produzione di olive per sostituire i dati simulati
- Esplorare modelli più avanzati, come le reti neurali o i modelli di ensemble
- Incorporare altri fattori che potrebbero influenzare la produzione, come le pratiche agricole o l'età degli alberi
- Sviluppare una dashboard interattiva basata su questo modello